
# Assignment 7: Decision Trees

_**NOTE**: this assignment is only mandatory for students registered for CSC 8515; students registered for CSC 4515 are **not** required to complete this assignment._

For this assignment, your goal is to implement the ID3 algorithm (greedy information-gain based decision tree induction).  You should use the Congressional Voting Records data set, the same one that we used with Naive Bayes, and for the same reason; it is a discrete multivariate dataset, which is well suited to decision trees.  

Note that while scikit-learn does have decision trees, it doesn't really handle discrete data well, so the library version may not produce exactly the same trees that your algorithm will.

Implement and test your algorithm; be sure you can print the actual tree (a simple text-based visualization is fine), as well as calculating classification accuracy on held-out testing data.


In [39]:
import math


class Tree:
    def __init__(self, parent=None):
        self.parent = parent
        self.children = []
        self.splitFeature = None
        self.splitFeatureValue = None
        self.label = None


def dataToDistribution(data):
    ''' Turn a dataset which has n possible classification labels into a
        probability distribution with n entries. '''
    allLabels = [label for (point, label) in data]
    numEntries = len(allLabels)
    possibleLabels = set(allLabels)

    dist = []
    for aLabel in possibleLabels:
        dist.append(float(allLabels.count(aLabel)) / numEntries)

    return dist


def entropy(dist):
    ''' Compute the Shannon entropy of the given probability distribution. '''
    return -sum([p * math.log(p, 2) for p in dist])


def splitData(data, featureIndex):
    ''' Iterate over the subsets of data corresponding to each value
        of the feature at the index featureIndex. '''

    # get possible values of the given feature
    attrValues = [point[featureIndex] for (point, label) in data]

    for aValue in set(attrValues):
        dataSubset = [(point, label) for (point, label) in data
                      if point[featureIndex] == aValue]

        yield dataSubset


def gain(data, featureIndex):
    ''' Compute the expected gain from splitting the data along all possible
        values of feature. '''

    entropyGain = entropy(dataToDistribution(data))

    for dataSubset in splitData(data, featureIndex):
        entropyGain -= entropy(dataToDistribution(dataSubset))

    return entropyGain


def homogeneous(data):
    ''' Return True if the data have the same label, and False otherwise. '''
    return len(set([label for (point, label) in data])) <= 1


def majorityVote(data, node):
    ''' Label node with the majority of the class labels in the given data set. '''
    labels = [label for (pt, label) in data]
    choice = max(set(labels), key=labels.count)
    node.label = choice
    return node


def buildDecisionTree(data, root, remainingFeatures):
    ''' Build a decision tree from the given data, appending the children
        to the given root node (which may be the root of a subtree). '''

    if homogeneous(data):
        root.label = data[0][1]
        return root

    if len(remainingFeatures) == 0:
        return majorityVote(data, root)

    # find the index of the best feature to split on
    bestFeature = max(remainingFeatures, key=lambda index: gain(data, index))

    if gain(data, bestFeature) == 0:
        return majorityVote(data, root)

    root.splitFeature = bestFeature
    for dataSubset in splitData(data, bestFeature):
        aChild = Tree(parent=root)
        aChild.splitFeatureValue = dataSubset[0][0][bestFeature]
        root.children.append(aChild)

        buildDecisionTree(dataSubset, aChild,
                          remainingFeatures - set([bestFeature]))

    return root


def decisionTree(data):
    return buildDecisionTree(data, Tree(), set(range(len(data[0][0]))))


def classify(tree, point):
    ''' Classify a data point by traversing the given decision tree. '''

    if tree.children == []:
        return tree.label
    else:
        matchingChildren = [child for child in tree.children
                            if child.splitFeatureValue == point[tree.splitFeature]]

    child = Tree()
    if matchingChildren:
        child = matchingChildren[0]
    return classify(child, point)

def classifyNoisy(tree, point):
    ''' Classify a noisy data point by traversing the given decision tree.
       Return a dictionary of the appropriate class counts to account for
       multiple branching. '''

    if tree.children == []:
        return tree.classCounts
    elif point[tree.splitFeature] == '?':
        dicts = [classifyNoisy(child, point) for child in tree.children]
        return dictionarySum(*dicts)
    else:
        matchingChildren = [child for child in tree.children
        if child.splitFeatureValue == point[tree.splitFeature]]
            return classifyNoisy(matchingChildren[0], point)


def classify2(tree, point):
   ''' Classify data which is assumed to have the possibility of being noisy.
       Return the label corresponding to the maximum among all possible
       continuations of the tree traversal. That is, the maximum of the
       class counts at the leaves. Classify2 is equivalent to classify
       if the data are not noisy. If the data are noisy, classify will
       raise an error. '''
   counts = classifyNoisy(tree, point)

   if len(counts.keys()) == 1:
      return counts.keys()[0]
   else:
      return max(counts.keys(), key=lambda k: counts[k])


def testClassification(data, tree):
    actualLabels = [label for point, label in data]
    predictedLabels = [classify(tree, point) for point, label in data]

    correctLabels = [(1 if a == b else 0) for a, b in zip(actualLabels, predictedLabels)]
    return float(sum(correctLabels)) / len(actualLabels)

def testTreeSize(noisyData, cleanData):
    import random

    for i in range(1, len(cleanData)):
        tree = decisionTree(random.sample(cleanData, i))
    print str(testClassification(noisyData, tree)) + ", ",

with open('house-votes-84.data', 'r') as inputFile:
    lines = inputFile.readlines()

data = [line.strip().split(',') for line in lines]
data = [(x[1:], x[0]) for x in data]

cleanData = [x for x in data if '?' not in x[0]]
noisyData = [x for x in data if '?' in x[0]]

# tree = decisionTree(cleanData)
tree = decisionTree(cleanData)
testClassification(cleanData, tree)

printBinaryDecisionTree(tree)
print(classify(tree, ['y' for _ in range(len(data[0][0]))]))
print(classify(tree, ['n' for _ in range(len(data[0][0]))]))


IndentationError: unexpected indent (<ipython-input-39-a65aee6580c9>, line 131)